❗ Оставлены рекомендации ревьюера

# Выбор локации для скважины

Вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Описание данных

* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

**Условия задачи:**
1. Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
2. При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
3. Бюджет на разработку скважин в регионе — 10 млрд рублей.
4. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
5. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

*Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.*


<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Прибыль" data-toc-modified-id="Прибыль-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Прибыль</a></span></li><li><span><a href="#Риски" data-toc-modified-id="Риски-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Риски</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import sklearn
import numpy as np



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


region_0 = pd.read_csv('/datasets/geo_data_0.csv')
region_1 = pd.read_csv('/datasets/geo_data_1.csv')
region_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
region_0.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [ ]:
region_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
region_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [ ]:
region_0.duplicated().sum()

0

В этой таблице пропусков и дубликатов нет, данные выглядят адекватно. Обработка не понадобится.

In [ ]:
region_1.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [ ]:
region_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
region_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [ ]:
region_1.duplicated().sum()

0

С этой таблицей тоже всё хорошо. Кстати, можем заметить, что есть скважины совсем без сырья.

In [ ]:
region_2.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [ ]:
region_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
region_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [ ]:
region_2.duplicated().sum()

0

И тут всё нормально. С типами данных тоже порядок, ничего заменять не требуется.
  
Так как в признаках у нас встречаются и отрицательные, и положительные значения разной величины, то можно сделать вывод, что  признаки уже масштабированы.

## Обучение и проверка модели

Разделим данные на признаки и целевой признак

In [ ]:
feature_names = ['f0', 'f1', 'f2']

features_0 = region_0[feature_names]
target_0 = region_0['product']
features_1 = region_1[feature_names]
target_1 = region_1['product']
features_2 = region_2[feature_names]
target_2 = region_2['product']

Разделим данные на обучающую и валидационную выборки

In [ ]:
features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

Обучим модель линейной регресии для каждого региона, выведем оценку RMSE и средний запас предсказанного сырья в месторождении.

In [ ]:
model_lr_0 = LinearRegression().fit(features_0_train, target_0_train)


predictions_0 = model_lr_0.predict(features_0_valid)


print('RMSE:','{:.3f}'.format(mean_squared_error(target_0_valid,predictions_0)**0.5))
print('Cредний запас предсказанного сырья: {:.3f}'.format(predictions_0.mean()))

RMSE: 37.579
Cредний запас предсказанного сырья: 92.593


In [ ]:
model_lr_1 = LinearRegression().fit(features_1_train, target_1_train)


predictions_1 = model_lr_1.predict(features_1_valid)


print('RMSE:','{:.3f}'.format(mean_squared_error(target_1_valid,predictions_1)**0.5))
print('Cредний запас предсказанного сырья: {:.3f}'.format(predictions_1.mean()))

RMSE: 0.893
Cредний запас предсказанного сырья: 68.729


In [ ]:
model_lr_2 = LinearRegression().fit(features_2_train, target_2_train)


predictions_2 = model_lr_2.predict(features_2_valid)


print('RMSE:','{:.3f}'.format(mean_squared_error(target_2_valid,predictions_2)**0.5))
print('Cредний запас предсказанного сырья: {:.3f}'.format(predictions_2.mean()))

RMSE: 40.030
Cредний запас предсказанного сырья: 94.965


Сразу бросается в глаза, что у второго региона RMSE стремится к нулю. Это говорит о точности предсказаний и качестве построенной модели. Однако средний запас нефти в этом регионе меньше почти на 30 тыс. баррелей, чем в остальных.

## Подготовка к расчёту прибыли

**Задачи:**

1. Рассчитать достаточный объём сырья для безубыточной разработки новой скважины.
2. Сравнить полученный объём сырья со средним запасом в каждом регионе.
3. Написать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

Сохраним в отдельных переменных все ключевые значения для расчета прибыли и рисков.

*budget* — Бюджет на разработку скважин в регионе — 10 млрд рублей  
*top_wells* — 200 лучших скважин в регионе  
*oil_wells* — выборка из 500 скважин в регионе  
*unit_income* — доход с каждой единицы продукта  

In [ ]:
budget = 10**10
top_wells = 200
oil_wells = 500
unit_income = 450000

<div class="alert alert-warning">
<b>⚠️ Комментарий ревьювера:</b> Константы принято записывать заглавными буквами. Это общепринятый стандарт у программистов:
    
https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html
    
Для удобства расчет можно вести в млн. рублей.
</div>

Дальше посчитаем минимальный объём продукта с одной скважины для безубыточной разработки. Для этого мы посчитаем, сколько денег минимально получим с лучших скважин и разделим на эту сумму сумму всего бюджета.

In [ ]:
minimal_size = budget / (top_wells * unit_income)
minimal_size

111.11111111111111

### Вывод

1. Минимальный объём продукта для безубыточной разработки составляет 111.11 тыс. баррелей.
2. Это значение превышает средние значения запасов скважин по всем регионам.

## Расчёт прибыли и рисков

### Прибыль

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

1. Выберем скважины с максимальными значениями предсказаний. Количество скважин зависит от бюджета компании и стоимости разработки одной скважины. В нашем случае выберем случайно 500 точек и отберём 200 с максимальным показателем объёма сырья
2. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
3. Рассчитаем прибыль для полученного объёма сырья.

In [ ]:
target_0_valid = pd.Series(target_0_valid).reset_index(drop=True)
target_1_valid = pd.Series(target_1_valid).reset_index(drop=True)
target_2_valid = pd.Series(target_2_valid).reset_index(drop=True)

In [ ]:
predictions_0 = pd.Series(predictions_0)
predictions_1 = pd.Series(predictions_1)
predictions_2 = pd.Series(predictions_2)

Функция берёт выборку размера oil_wells, выбирает top_wells из самых больших значений и считает суммарный целевой объём сырья и прибыль для полученного объёма сырья.  
Если show_res=True, то функция выведет текст с результатами, а если False, то просто вернёт значение прибыли. Результаты мы выведем в этом пункте, а просто прибыль нам будет нужна при подсчёте рисков

In [ ]:
state = 12345
def income(target, predictions, region, state, show_res=True):

    sample_preds = predictions.sample(n=oil_wells, random_state=state)
    top_preds = sample_preds.sort_values(ascending=False)[:top_wells]
    top_targets = target[top_preds.index]
    volume = sum(top_targets)
    income = volume * unit_income - budget
    if show_res:
        print(region)
        print('Суммарный целевой объём сырья: {:.2f}'.format(volume))
        print('Прибыль для полученного объёма сырья: {:.2f}'.format(income))
    else:
        return income

In [ ]:
income(target_0_valid, predictions_0, 'Регион 0', state, show_res=True)

Регион 0
Суммарный целевой объём сырья: 23731.26
Прибыль для полученного объёма сырья: 679068857.89


In [ ]:
income(target_1_valid, predictions_1, 'Регион 1', state, show_res=True)

Регион 1
Суммарный целевой объём сырья: 23954.40
Прибыль для полученного объёма сырья: 779479884.18


In [ ]:
income(target_2_valid, predictions_2, 'Регион 2', state, show_res=True)

Регион 2
Суммарный целевой объём сырья: 23199.98
Прибыль для полученного объёма сырья: 439990143.02


### Риски

**Задачи:**

1. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
2. Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [ ]:
def bootstrap(predictions, region, target):
    state = np.random.RandomState(12345)
    values = []


    for i in range(1000):
        #income = income(target, predicts, region, state, replace=True, show_res=False)
        values.append(income(target, predictions, region, state, show_res=False))

    # находим доверительный интервал и риск убытков
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk = len(values[values<0])/len(values)

    print(region)
    print('Средняя прибыль:','{:.2f}'.format(values.mean()))
    print('Начало доверительного интервала:','{:.4f}'.format(lower))
    print('Конец доверительного интервала:','{:.4f}'.format(upper))
    print('Риск убытков: {:.2f} %'.format(risk * 100))

In [ ]:
values = bootstrap(predictions_0, 'Регион 0', target_0_valid)

Регион 0
Средняя прибыль: 380710890.71
Начало доверительного интервала: -126947638.0318
Конец доверительного интервала: 879613967.8478
Риск убытков: 7.20 %


In [ ]:
values = bootstrap(predictions_1, 'Регион 1', target_1_valid)

Регион 1
Средняя прибыль: 448231065.15
Начало доверительного интервала: 70899384.9354
Конец доверительного интервала: 892985249.7001
Риск убытков: 1.40 %


In [ ]:
values = bootstrap(predictions_2, 'Регион 2', target_2_valid)

Регион 2
Средняя прибыль: 402796587.17
Начало доверительного интервала: -143659306.8483
Конец доверительного интервала: 963026154.4863
Риск убытков: 7.10 %


<div class="alert alert-success">
<b> ✔️ Комментарий ревьювера:</b>
    
Вот небольшая статья по бутстрэпу в том числе, вдруг заинтересует:

https://habr.com/ru/company/ods/blog/324402/
    
</div>

## Вывод

Бурить скважину лучше в регионе 1, так как:

1. Из всех регионов максимальная прибыль именно в нём (448.23 млн против 380 млн и 402 млн).
2. Только регион 1 прогнозирует прибыльную разработку по 95% доверительному интервалу.
3. Риск убытков для региона 1 самый минимальный — 1.4%, он почти на 6% ниже, чем в 0 и 2 регионах.
